In [2]:
import pandas as pd
import pandas as pd
from sklearn import preprocessing
from sklearn import metrics
import numpy as np

In [3]:
label = pd.read_csv('../../data/comp5046-labels.csv',header=None)
Lr_label = list(set(label[2].values))


def prediction(x):
    tp = []
    for w in W:
        tp.append(x.dot(w))
    predict_num = tp.index(max(tp))
    return predict_num

folds_number = int(len(label)*0.1)+1
num_of_epoch = 5
num_of_x = 99688
num_of_chunksize = folds_number
address_x = 'result/two_NN_data.csv'
address_label = '../../data/comp5046-labels.csv'

folds = list(range(10))

for fold in folds:
    step = (num_of_epoch*(len(label)-folds_number))
    total_step = step
    
    W = [np.zeros(num_of_x) for i in range(len(Lr_label))]
    a = [np.zeros(num_of_x) for i in range(len(Lr_label))] # averaged w
    for epoch in range(num_of_epoch):
        number_of_training = 0
        predict_number_list = list(range(folds_number * fold, folds_number * (fold + 1)))
        for text in pd.read_csv(address_x,header=None,chunksize=num_of_chunksize):
            label = pd.read_csv(address_label,header=None)        
            text = pd.merge(text,label[[1,2]],left_on=num_of_x,right_on=1)
            y = [Lr_label.index(it) for it in text['2_y'].values]
            X = text.values[:,:-3]
            for each_data in range(len(X)):
                if number_of_training not in predict_number_list:
                    number_of_training+=1
                    y_ = prediction(X[each_data])
                    true_y = y[each_data]
                    step -=1
                    if y_ != true_y:
                        W[true_y] = W[true_y] + X[each_data]
                        W[y_] = W[y_] - X[each_data]
                        a[true_y] = a[true_y] + (X[each_data])*(step/(num_of_epoch*len(label)-folds_number))
                        a[y_] = a[y_] - (X[each_data])*(step/(num_of_epoch*len(label)-folds_number))
                        print('#'*(100-int(100*step/total_step))+'-'*int((step/total_step)*100)+' '+str(int(100-(step/total_step)*100))+'%',end='\r',flush=True)
#                     
                else:
                    number_of_training+=1
#         print('epoch: ',epoch,'Finished trainig.')
        
    W = a  
    number_of_training = 0
    total_y_ = []
    total_y = []

    total_y_label = [Lr_label[i] for i in total_y]
    total_y_label_ = [Lr_label[i] for i in total_y_]
    predict_number_list = list(range(folds_number * fold, folds_number * (fold + 1)))
    for text in pd.read_csv(address_x,header=None,chunksize=num_of_chunksize):
        label = pd.read_csv(address_label,header=None)
        text = pd.merge(text,label[[1,2]],left_on=num_of_x,right_on=1)

        y = [Lr_label.index(it) for it in text['2_y'].values]

        X = text.values[:,:-3]
        for each_data in range(len(X)):

            if number_of_training in predict_number_list and number_of_training < folds_number* (fold + 1):
                number_of_training+=1
                y_ = prediction(X[each_data])
                true_y = y[each_data]

                total_y_.append(y_)
                total_y.append(true_y)
            elif number_of_training not in predict_number_list and number_of_training < folds_number * fold:
                number_of_training+=1
            elif number_of_training not in predict_number_list and number_of_training > folds_number * (fold + 1):
                number_of_training+=1
                print('Testing is over.')
                break

        if len(total_y_) == folds_number: # Save the perfomance of result
            total_y_label = [Lr_label[i] for i in total_y]
            total_y_label_ = [Lr_label[i] for i in total_y_]
            print("Prediction score:\n%s" % metrics.classification_report(total_y_label, total_y_label_))
            result = metrics.classification_report(total_y_label, total_y_label_)
            macro = ' '.join(list(map(str,metrics.precision_recall_fscore_support(total_y_label,total_y_label_,average='macro'))))
            micro = ' '.join(list(map(str,metrics.precision_recall_fscore_support(total_y_label,total_y_label_,average='micro'))))
            with open('result_%s.csv' % fold, 'w') as file:  # Save the metrics result to file
                for line in result:
                    file.write(line)
#                 elapsed_time = time.time() - start_time
#                 file.write('Fold %s, time is %s. \n' % (fold, elapsed_time))  # Save the time cost to file
                file.write('\n macro: '+macro)
                file.write('\n micro: '+micro)
            break



/root/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Prediction score:
                       precision    recall  f1-score   support

            Education       0.51      0.83      0.63        24
        Entertainment       0.79      0.70      0.74        87
                Error       0.30      0.31      0.31        32
              Finance       0.50      0.33      0.40         3
               Health       0.09      1.00      0.17         1
          Hospitality       0.00      0.00      0.00         2
                Other       0.00      0.00      0.00         3
Public administration       0.47      0.42      0.44        33
          Real estate       0.00      0.00      0.00         2
            Resources       0.67      1.00      0.80         2
              SciTech       0.40      0.40      0.40         5
               Social       0.46      0.50      0.48        34
               Sports       0.70      0.64      0.67        44
                Trade       0.20      0.12      0.15        16
            Transport       0.44    

/root/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Prediction score:
                       precision    recall  f1-score   support

            Education       0.56      0.77      0.65        31
        Entertainment       0.84      0.66      0.74        74
                Error       0.14      0.18      0.15        17
              Finance       0.40      0.67      0.50         3
               Health       0.00      0.00      0.00         0
          Hospitality       0.00      0.00      0.00         2
                Other       0.00      0.00      0.00         2
Public administration       0.65      0.45      0.53        38
          Real estate       0.00      0.00      0.00         1
              SciTech       0.33      0.20      0.25        10
               Social       0.40      0.45      0.42        31
               Sports       0.94      0.76      0.84        63
                Trade       0.40      0.47      0.43        17
            Transport       0.33      0.44      0.38         9

          avg / total       0.65   